## Connection

In [1]:
# libraries import
import pandas as pd
from sqlalchemy import create_engine
# set parameters for connection
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp', 
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, 
 'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# save connection
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Primary Data Analysis

In [2]:
# select first 5 rows for each table
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print(
        f'\n\033[1;30;47m_____________SET:  {table.upper()}_____________\033[0;0m\n')
    display(pd.io.sql.read_sql(f'''SELECT * FROM {table} LIMIT 5''', con = engine))


_____________SET:  BOOKS_____________



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



_____________SET:  AUTHORS_____________



,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



_____________SET:  PUBLISHERS_____________



,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



_____________SET:  RATINGS_____________



,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



_____________SET:  REVIEWS_____________



,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Task №1
Count how many books have been published since January 1, 2000.

In [3]:
query = '''
SELECT 
  COUNT(book_id) 
FROM 
  books 
WHERE 
  publication_date > '2000-01-01';
'''

In [4]:
pd.io.sql.read_sql(query, con = engine)

,count
0,819


*Conclusions: 819 books were published after January 1, 2000.*

## Task №2
For each book, count the number of reviews and calculate the average rating.

In [5]:
query = '''
SELECT 
    books.title, 
    COUNT(distinct reviews.review_id) AS reviews_cnt, 
    AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY 
    books.book_id
ORDER BY 
    COUNT(reviews.review_id) DESC
LIMIT 10
'''

In [6]:
pd.io.sql.read_sql(query, con = engine)

,title,reviews_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
5,Angels & Demons (Robert Langdon #1),5,3.678571
6,Harry Potter and the Order of the Phoenix (Har...,5,4.186667
7,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
8,Animal Farm,5,3.729730
9,The Fellowship of the Ring (The Lord of the Ri...,5,4.391892


*Conclusions: Twilight is the leader in terms of reviews (7 reviews), with an average rating of 3.66).*

## Task №3
Determine the publisher that has released the most books thicker than 50 pages - this way you will exclude brochures from the analysis.

In [7]:
query = '''
SELECT
    publisher,
    COUNT(book_id) as book_cnt
FROM
    publishers
LEFT JOIN books ON  books.publisher_id = publishers.publisher_id 
WHERE 
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    book_cnt DESC
LIMIT 10
'''

In [8]:
pd.io.sql.read_sql(query, con = engine)

,publisher,book_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


*Conclusions: the largest number of books was published by Penguin Books - 42 books.*

## Task №4
Determine the author with the highest average rating of books - consider only books with 50 or more ratings.

In [9]:
query = '''
SELECT
    author,
    AVG(sub.avg_rating) AS high_avg_rating
FROM
    (SELECT
        author,
        books.book_id,
        AVG(rating) AS avg_rating
    FROM 
        authors
    LEFT JOIN books ON books.author_id = authors.author_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        author,
        books.book_id
    HAVING
        COUNT(rating) > 50) AS sub
GROUP BY
    author
ORDER BY
    high_avg_rating DESC
'''

In [10]:
pd.io.sql.read_sql(query, con = engine)

,author,high_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


*Conclusions: the author with the highest average rating of books is J.K. Rowling/Mary GrandPré (4.28).*

## Task №5
Calculate the average number of reviews from users who have given more than 50 ratings.

In [11]:
query = '''
SELECT
    AVG(sub_review_cnt.review_cnt) AS avg_review_cnt
FROM
    (SELECT
        COUNT(text) AS review_cnt
    FROM 
        reviews
    WHERE username IN
        (SELECT username
        FROM 
            ratings
        GROUP BY
            username
        HAVING
            COUNT(ratings) > 50)
    GROUP BY 
            username
    ) AS sub_review_cnt  
'''

In [12]:
pd.io.sql.read_sql(query, con = engine)

,avg_review_cnt
0,24.333333


*Conclusions: the average number of reviews from users is 24.3.*

# General conslusion

<div style="border:solid green 2px; padding: 20px">
    
    
In this project, we analyzed a database of an online reader application - with information about books, publishers, authors, and user reviews of books. Based on the queries, the following conclusions were made:

- 819 books were released after January 1, 2000;
- The leader in terms of reviews was Twilight (7 reviews, average rating 3.66);
- The publisher with the most books released was Penguin Books - 42 books;
- The author with the highest average rating of books was J.K. Rowling/Mary GrandPré (4.28);
- The average number of user reviews was 24.3.